In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelcheckpointConfig:
    root_door: Path
    model_save: Path
    monitor: str
    save_best_only: bool
    save_weights_only: bool
    verbose: int
    patience: int
    restore_best_weights: bool
    factor: float
    min_lr: float

In [7]:
from src.constants import *
from src.utils.common import read_yaml,create_directories

In [16]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.data_root])

    def get_model_checkpoint_config(self) -> ModelcheckpointConfig:
        config = self.config.Model_Checkpoint
        create_directories([config.root_dir])

        modelcheckpoint_config = ModelcheckpointConfig(
            root_door = config.root_dir,
            model_save= config.model_save,
            monitor = self.params['Model_Checkpoint']['monitor'],
            save_best_only = self.params['Model_Checkpoint']['save_best_only'],
            save_weights_only = self.params['Model_Checkpoint']['save_weights_only'],
            verbose = self.params['Model_Checkpoint']['verbose'],
            patience = self.params['Early_stopping']['patience'],
            restore_best_weights = self.params['Early_stopping']['restore_best_weights'],
            factor= self.params['ReduceLROnPlateau']['factor'],
            min_lr= self.params['ReduceLROnPlateau']['min_lr']
        )

        return modelcheckpoint_config

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [24]:
class ModelCheckpointManager:
    def __init__(self,config:ModelcheckpointConfig):
        self.config = config

    def get_the_model_chceckpoint(self):
        model_check = ModelCheckpoint(
        filepath = self.config.model_save,
        monitor = self.config.monitor,
        verbose = self.config.verbose,
        save_best_only = self.config.save_best_only,
        save_weights_only=self.config.save_weights_only

    )

        early_stopping = EarlyStopping(
            monitor = self.config.monitor,
            patience =self.config.patience,
            verbose = self.config.verbose,
            restore_best_weights = self.config.restore_best_weights
        )

        reduce_lr = ReduceLROnPlateau(
            monitor=self.config.monitor,
            factor=self.config.factor,
            patience=self.config.patience,
            min_lr=self.config.min_lr,
    )
        # Print the ModelCheckpoint variables
        print("Model Checkpoint Configuration:")
        print(f"Filepath: {self.config.model_save}")
        print(f"Monitor: {self.config.monitor}")
        print(f"Verbose: {self.config.verbose}")
        print(f"Save Best Only: {self.config.save_best_only}")
        print(f"Save Weights Only: {self.config.save_weights_only}")

        # Print the EarlyStopping variables
        print("\nEarly Stopping Configuration:")
        print(f"Monitor: {self.config.monitor}")
        print(f"Patience: {self.config.patience}")
        print(f"Verbose: {self.config.verbose}")
        print(f"Restore Best Weights: {self.config.restore_best_weights}")

        # Print the ReduceLROnPlateau variables
        print("\nReduce Learning Rate Configuration:")
        print(f"Monitor: {self.config.monitor}")
        print(f"Factor: {self.config.factor}")
        print(f"Patience: {self.config.patience}")
        print(f"Min LR: {self.config.min_lr}")


        return model_check, early_stopping, reduce_lr

In [19]:
from tensorflow.keras.models import load_model

In [20]:
model = load_model('data/model_building/Chicken_fecal.keras')

/Applications/anaconda3/envs/chkn/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
try:
    config = ConfigurationManager()
    chk_config = config.get_model_checkpoint_config()
    model_checkpoint = ModelCheckpointManager(config=chk_config)
    model_check, early_stopping, reduce_lr = model_checkpoint.get_the_model_chceckpoint()
    print(model_check)
    print("No issue with code")
except Exception as e:
    raise e

2024-09-20 21:34:48,346 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-20 21:34:48,349 - INFO - Yaml file: params.yaml loaded successfully
2024-09-20 21:34:48,350 - INFO - created directory at: data
2024-09-20 21:34:48,353 - INFO - created directory at: data/model_checkpoint
Model Checkpoint Configuration:
Filepath: data/model_checkpoint/Chicken_fecal.keras
Monitor: accuracy
Verbose: 1
Save Best Only: True
Save Weights Only: False

Early Stopping Configuration:
Monitor: accuracy
Patience: 2
Verbose: 1
Restore Best Weights: True

Reduce Learning Rate Configuration:
Monitor: accuracy
Factor: 0.2
Patience: 2
Min LR: 0.0001
No issue with code
